In [1]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit import Aer
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
#from qiskit import register, available_backends, get_backend
from math import pi
#from qiskit.wrapper.jupyter import *
import matplotlib.pyplot as plt
import sys
global ctg
from qiskit.tools.visualization import circuit_drawer
from parseRealization import *
%matplotlib inline

IBMQ.load_accounts()

testDir="./tests/"

C:\Users\krylo\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


In [2]:
def flipTheString(s):
    myString=""
    length = len(s)
    for it in range(0,length):
        myString=myString+s[length-1-it]
    return myString

def quasmToMatlab(s):
    count = 0
    lines = s.split('\n')
    for line in lines:
        if line.startswith("x"):
            count = count + 1
            tokens = int(line.split(" ")[1].split("[")[1].split("]")[0])+1
            string = str(tokens)+" "+str(tokens)+" 0 "+str(pi)+" "+str(0)
            print(string)
        if line.startswith("swap"):
            count = count + 1
            tokens = line.split("[")
            index1 = int(tokens[1].split("]")[0])+1
            index2 = int(tokens[2].split("]")[0])+1
            string = str(index1)+" "+str(index2)+" 2 "+str(-10)+" "+str(0)
            print(string)
        if line.startswith("cx"):
            count = count + 1
            tokens = line.split("[")
            index1 = int(tokens[1].split("]")[0])+1
            index2 = int(tokens[2].split("]")[0])+1
            q = -15
            if index1>index2:
                q = -20
                t = index1
                index1 = index2
                index2 = t
            string = str(index1)+" "+str(index2)+" 2 "+str(q)+" "+str(0)
            print(string)
        if line.startswith("ry"):
            count = count + 1
            tokens = line.split("(")
            parameter = float(tokens[1].split(")")[0])
            index = int(tokens[1].split("[")[1].split("]")[0])+1
            string = str(index)+" "+str(index)+" 1 "+str(parameter)+" "+str(0)
            print(string)
        if line.startswith("rzz"):
            count = count + 1
            tokens = line.split("(")
            parameter = float(tokens[1].split(")")[0])
            tokenz = tokens[1].split("[")
            index1 = int(tokenz[1].split("]")[0])+1
            index2 = int(tokenz[2].split("]")[0])+1
            
            string = str(index1)+" "+str(index2)+" 2 "+str(parameter)+" "+str(0)
            print(string)
        if line.startswith("rz("):
            count = count + 1
            tokens = line.split("(")
            parameter = float(tokens[1].split(")")[0])
            index = int(tokens[1].split("[")[1].split("]")[0])+1
            string = str(index)+" "+str(index)+" 2 "+str(parameter)+" "+str(0)
            print(string)
        if line.startswith("rx"):
            count = count + 1
            tokens = line.split("(")
            parameter = float(tokens[1].split(")")[0])
            index = int(tokens[1].split("[")[1].split("]")[0])+1
            string = str(index)+" "+str(index)+" 0 "+str(parameter)+" "+str(0)
            print(string)
    print(count)


def testFromFile(fileName,details=0):
    ioClass = readCircuitInformation(fileName) 
    answers = ioClass.getKmap()
    errors=[]
    size = ioClass.getSize()
    ioClass.readGatesFromFile(fileName)
    print (ioClass.getKmap())
    error_count = 0
    backend_sim = Aer.get_backend('qasm_simulator')
    t = 0
    for i in answers:
        ctg = CircuitTransitionGraph()
        ctg.setSize(size)
        qr = QuantumRegister(size)
        cr = ClassicalRegister(size)
        qc = QuantumCircuit(qr,cr)
        for j in range(0,size):
            if i[j]=="1":
                qc.x(qr[j])
        qc,qr = ctg.readGatesFromIOClass(qc,qr,ioClass)

        qc.measure(qr,cr)
        if t == 0 and details == 1:
            circuit_drawer(qc).show()
            print(qc.qasm())
            t = 1
        job_sim = execute(qc, backend_sim)#,config)
        stats_sim = job_sim.result().get_counts()
        if details==1:
            # circuit_drawer(qc) won't work, why though
            print(i,stats_sim)
        for release in stats_sim:
            myString = flipTheString(release)
            if myString != answers[i]:
                error = i + ":" + myString + ":"+answers[i]
                errors = errors + [error]
                error_count = error_count+1
    print(ctg)
    if error_count != 0 :
        print("FAILURE:",fileName)
    if details==1:
        #print(qc.qasm())
        #quasmToMatlab(qc.qasm())
        print("Input:Received:Expected")
        for item in errors:
            print(item)
        print(answers)
        print("ERROR:",error_count)
    return error_count
            #plot_histogram(stats_sim)

In [3]:
#test not

testNames=["test00","testCVCVdag","ham3_28","miller","hwb4_52","testVCVC","3_17","test01","simpleSwap","swap1-2","swap1-3", "swap3-2","test1","testCVCV","simpleToffoli","hwb4_52_mod","toffoli"]
fails = 0
for fileName in testNames:
    fileName = testDir + fileName 
    fails = fails + testFromFile(fileName,1)
if fails == 0:
    print("SUCCESS")
print(fails,len(testNames))
    

./tests/testCVCVdag.pla
{'00': '00', '01': '01', '10': '10', '11': '11'}


C:\Users\krylo\Anaconda3\lib\site-packages\qiskit\tools\visualization\_circuit_visualization.py:206: DeprecationWarning: The current behavior for the default output will change in a future release. Instead of trying latex and falling back to mpl on failure it will just use "text" by default
  '"text" by default', DeprecationWarning)


OPENQASM 2.0;
include "qelib1.inc";
qreg q0[2];
creg c0[2];
h q0[1];
cx q0[1],q0[0];
t q0[1];
tdg q0[0];
cx q0[1],q0[0];
h q0[1];
t q0[0];
tdg q0[0];
h q0[1];
cx q0[1],q0[0];
t q0[0];
tdg q0[1];
cx q0[1],q0[0];
h q0[1];
measure q0[0] -> c0[0];
measure q0[1] -> c0[1];

00 {'00': 1024}
01 {'10': 1024}
10 {'01': 1024}
11 {'11': 1024}
Weights are:{'ab': 2}, skeleton is:['ab', 'ab'], vertices are{'a', 'b'}, paths are {},latest weight are[]
Input:Received:Expected
{'00': '00', '01': '01', '10': '10', '11': '11'}
ERROR: 0
SUCCESS
0 1


C:\Users\krylo\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning
